### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [1]:
# poetry add python-dotenv langchain langchain-openai

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_Z


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    #model="openai/gpt-oss-120b",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 인터넷의 방대한 텍스트에서 단어와 문장의 출현 패턴을 학습해, 다음에 올 가능성이 높은 토큰을 확률적으로 '
 '예측합니다.  \n'
 '학습 과정에서 인간의 피드백(RLHF)을 통해 유해하거나 부정확한 답변을 줄이고, 사용자의 선호에 맞는 대화 스타일을 강화합니다.  \n'
 '결국 통계적 패턴 매칭과 보상 최적화를 통해 “문맥에 맞는 말”을 만들어내는, 말하자면 “확률 기반의 자동완성 기계”입니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [4]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="한국어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"한국어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('1. ChatGPT는 인터넷의 방대한 글을 미리 학습해 “다음 단어를 예측하는” 방식으로, 사람이 쓴 글의 패턴을 숫자로 '
 '기억합니다.  \n'
 '2. 사용자가 질문을 던지면, 그 질문을 숫자로 바꿔 기억 속 패턴과 비교해 가장 적절한 다음 단어를 반복적으로 선택합니다.  \n'
 '3. 이 과정에서 사람의 피드백(강화학습)을 받아 거짓·편향·무례한 답변을 줄이고, 유용하고 자연스러운 문장을 만들도록 미세 '
 '조정됩니다.\n'
 '\n'
 'ChatGPT 모델의 주요 장점 요약  \n'
 '• 맥락 이해 능력: 긴 대화나 글을 읽고 흐름을 기억해 논리적이고 일관된 답변 생성  \n'
 '• 다양한 스타일 지원: 설명·요약·창작·코드 작성 등 짧은 지시어만으로도 원하는 형식·톤으로 출력  \n'
 '• 즉시 활용 가능: 별도 재학습 없이 즉석에서 새로운 주소, 언어, 분야에 대응해 실용적 답변 제공  \n'
 '• 사용자 피드백 반영: RLHF로 거짓·유해 답변을 줄여 신뢰도·안전성 향상  \n'
 '• 다국어·멀티모달 지원: 한국어를 포함한 50여 개 언어, 이미지·음성 입력까지 처리 가능(버전에 따라)  \n'
 '• 확장生태계: 플러그인·API·파인튜닝 기능으로 고객 서비스, 교육, 코딩 보조 등 산업 전반에 쉽게 통합\n'
 '\n'
 'ChatGPT와 비슷한 AI 모델  \n'
 '• 구글 제미나이(Gemini)  \n'
 '• 메타 뚜껑열기(Llama)  \n'
 '• 안트로픽 클레

#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [5]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "claude", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT-4는 방대한 텍스트 데이터를 바탕으로 단어·구문 간 확률 관계를 학습해, 주어진 앞단어들 다음에 올 최적의 토큰을 예측하는 '
 '방식으로 훈련됩니다.  \n'
 '학습 과정에서 인간 평가자의 선호도를 반영한 강화학습 기법(RLHF)이 추가로 적용되어, 유용하고 안전한 답변을 생성하도록 미세 '
 '조정됩니다.  \n'
 '이렇게 학습된 대규모 트랜스포머 모델은 주어진 프롬프트를 기반으로 이전에 본 적 없는 새로운 문장을 논리적으로 연결해 만들어냅니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini는 텍스트·이미지·오디오·코드 등 다양한 데이터를 한꺼번에 받아들이는 멀티모달 아키텍처를 기반으로, 인코더-디코더 구조에 '
 '전문가 혼합(MoE) 방식을 더해 희소 활성화로 효율을 높인다.  \n'
 '학습 초기엔 다음 토큰 예측과 대규모 대비학습으로 일반 지식을 익히고, 이후 지시 따르기·인간 피드백 강화학습(RLHF)을 통해 사람의 '
 '선호도에 맞춘 정렬을 수행한다.  \n'
 '체인-of-thought 프롬프트와 정책 최적화 기법을 반복해 추론 경로를 확장·세밀하게 만들어 수학·코딩 등 복잡한 문제를 단계적으로 '
 '해결하도록 훈련한다.  \n'
 '모델 크기에 따라 Nano·Pro·Ultra 등 여러 버전으로 배포해, 일정 수준의 성능은 유지하면서 계산 비용과 추론 속도를 균형 있게 '
 '조절한다.')
<class 'str'> claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude는 대규모 언어 모델로, 인터넷의 방대한 텍스트를 

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [6]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. \
     Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI.      Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
ChatGPT는 “어떻게 공부하는가?”보다 “어떻게 ‘공부한 것’을 활용하는가?”에 초점이 맞춰진 모델입니다.  
요컨대, **‘대규모 텍스트를 반복해 읽으며 다음 단어를 맞히는 놀이’를 엄청난 양으로 하여, 그 과정에서 언어의 패턴을 암기한 뒤, 사용자 질문에 맞춰 패턴을 꺼내 조합하는 방식**입니다. 아래에 4단계로 나눠 설명하겠습니다.

----------------------------------------
1. 전체 흐름 요약
1) Pre-training(사전학습)  
   → “다음 단어 맞히기” 반복으로 ‘일반 언어 능력’을 획득  
2) Supervised Fine-Tuning(SFT, 미세조정)  
   → 질문-답변 데이터로 ‘대화 형식’을 익힘  
3) Reward Model(RM) 학습  
   → ‘좋은 답’ vs ‘나쁜 답’을 가르쳐 주는 평가사를 만듦  
4) PPO(강화학습) 최적화  
   → 평가사의 점수를 높이는 방향으로 추가 학습  
----------------------------------------
2. 1단계: Pre-training(언어 모델 만들기)
• 목표  
  – 주어진 앞 문장(컨텍스트)를 보고 다음에 나올 토큰(단어/부분단어)을 확률로 예측  
• 데이터  
  – 웹, 책, 논문, 위키 등 수천억 토큰  
• 모델 구조  
  –

In [7]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
ChatGPT는 ‘생성형 사전 학습 트랜스포머’(Generative Pre-trained Transformer) 계열 모델로,  
“거대 언어 모델이 언어를 얻는 법 = 확률적으로 다음 토큰(next token)을 예측하는 연속성의 학습”이라는 단일 목표를 반복하면서 놀랄 만큼 다양한 지능적 행동이 나타나게 됩니다. 핵심 원리를 단계별로 정리하면 다음과 같습니다.

1. 데이터 준비  
   • 공개 웹, 책, 위키, 논문, 코드 등 방대한 텍스트를 수집·세정·중복 제거.  
   • 문장을 ‘토큰’(단어·부분 단어·문자) 단위로 잘라 5만~10만 개 규모의 어휘 사전(vocabulary)을 만듭니다.  
   • 각 토큰은 정수 ID로 바뀌고, 모델은 이 ID 시퀀스만 “보고” 학습합니다.

2. 모델 구조 – 트랜스포머 디코더  
   • 총 L개의 어텐션-피드포워드 블록이 쌓인 순차 생성 모델.  
   • 셀프-어텐션은 “지금까지 나온 토큰들을 동시에 참고하여 현재 위치의 표현을 갱신”하는 행렬 연산.  
   • 위치 인코딩(positional encoding)으로 단어 순서 정보를 보존.  
   •因果(인과) 마스크로 “미래 토큰은 보지 못하게” 함 → 자기 회귀(auto-regressive) 생성 가능.  
   • 파라미터 수 = 수십 억~수천 억(예: GPT-3 175B, GPT-4 추정 1.8T MoE).  
   • 행렬-벡터 연산만으로 다음 토큰 확률 P(token|context)를 출력.

3. 1단계: 사전 학습(Pre-training) – 수퍼비전 없음  
   목적 함수: 최대 우도(Maximum Likelihood)  
        L = Σ_t log P(w_t | w_1 … w_{t−1}; θ)  
   • 한 에폭당 수조 토큰을 1~2개월간 A100/H100 GPU 수천 대로 학습.  
   • 경사 소실/폭주를 막기 위해 LayerNorm, Residual, AdamW, AMP, BF16/FP1

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [8]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [9]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 사랑해.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [10]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계 행성(태양에서 멀어지는 순)

1. 수성  
   - 지름: 지구의 38%  
   - 특징: 표면 온도 변화 극심, 달보다 작음

2. 금성  
   - 지름: 지구 95%  
   - 특징: 이산화탄소 대기·온실효과로 460 °C, 밤낮 구분 안 됨

3. 지구  
   - 유일한 액체 해양·생명 존재

4. 화성  
   - 지름: 지구 53%  
   - 특징: 적색 사막·극冠, 과거 유수 흔적, 위성 2개

5. 목성  
   - 질량: 다른 행성 합친 것의 2배  
   - 특징: 대폭풍(적점), 위성 90여 개(이오·가니메데 등)

6. 토성  
   - 특징: 현저한 고리, 위성 80여 개(티탄·엔켈라도스)

7. 천왕성  
   - 특징: 옆으로 누운 자전축(98°), 고리·위성 27개

8. 해왕성  
   - 특징: 초음속 바람, 위성 14개(트리톤)

※ 화성과 목성 사이에 ‘소행성대’, 해왕성 궤도 너머에 ‘카이퍼대’ 존재


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [11]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(llm)
print(chain)

# 테스트 실행
result = chain.invoke({"input": "양자컴퓨팅에 대하여 설명해 주세요."})
print(result.content)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000204E9DFF110> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000204E9E71280> root_client=<openai.OpenAI object at 0x00000204E98B43B0> root_async_client=<openai.AsyncOpenAI object at 0x00000204E9E3D1F0> model_name='moonshotai/kimi-k2-instruct-0905' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**:

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [12]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? \
        {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요?         가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을에 일어나는 대표적인 지구과학 현상은 **태풍 발생**이 맞긴 하지만, 태풍은 **여름~가을**에 걸쳐 발생하며, **가을에 특히 빈번**해지는 편입니다. 하지만 **가을에만** 발생하는 것은 아니에요.

---

### ✅ 가을에 **주로** 일어나는 지구과학 현상 3가지:

1. **시차현상(Equinox)**  
   - 9월 22일~23일경 발생하는 **추분(秋分)**으로, 낮과 밤의 길이가 같아지는 현상입니다.  
   - 지구의 자전축이 태양에 수직이 되는 시점.

2. **성층권 돌풍(SSW: Sudden Stratospheric Warming)**  
   - 북반구에서 겨울 전환기(10~11월)에 가까워지며 발생하는 **성층권의 급격한 온도 상승** 현상.  
   - 이후 북극 진동(AO) 약화로 인해 **한파**나 **이상 기후**로 이어질 수 있음.

3. **가을 태풍(秋台)**  
   - 9~10월에 집중 발생하는 태풍.  
   - 해수면 온도가 여름만큼 높고, 제트기류가 남하하면서 **강력한 태풍**이 자주 발생.  
   - 한국, 일본, 중국 동해안에 큰 피해를 줌.

---

### 요약
- **태풍**은 가을의 대표 현상 중 하나지만, **가을에만** 발생하는 건 아닙니다.  
- **추분**, **SSW**, **가을 태풍**이 가을에 **특히 뚜렷하게 나타나는** 지구과학 현상입니다.


In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season_name = get_current_season()  # 계절 값 얻기
print(f"현재 계절: {season_name}")

현재 계절: 가을


In [14]:

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
# llm = ChatOpenAI(
#     #api_key=OPENAI_API_KEY,
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
#     temperature=0.0
# )

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season_name}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season_name}에 발생하는 자연 현상:\n{response}")


 가을에 발생하는 자연 현상:
가을철에 뚜렷하게 나타나는 대표적인 지구과학 현상 3가지는 다음과 같습니다.

1. 시베리아 고기압 확장  
   아시아 대륙의 시베리아 상공에서 발달한 한랭고기압이 점차 세력을 키워 동아시아 쪽으로 확장합니다. 이로 인해 중국 북부·한반도·일본에 맑고 건조한 날씨가 이어지면서 기온이 빠르게 떨어지고, 우리나라에도 ‘쌀쌀한 가을바람’과 ‘일교차’가 뚜렷해집니다.

2. 산악 지형에서의 온대 저기압(가을장마)  
   여름철 몬숨이 약해지면서 북태평양 고기압이 남하하고, 그 사이에서 중국 대륙을 따라 동진하는 단파 저기압이 빈번히 형성됩니다. 특히 9~10월 한반도 산악지역(태백·소백산맥)에서는 지형 효과가 더해져 집중호우가 발생하는데, 이를 흔히 ‘가을장마’라 부릅니다.

3. 북극 진동(아크틱 오실레이션)의 양상 전환  
   북극과 중위도 사이의 기압차가 커지는 음의 진동(NAO·AO)이 늦가을에 강화되면 제트기류이 북상하고, 이로 인해 한반도에 한파가 잦아지며 겨울철 한파의 전조 현상으로 나타납니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [15]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1386.92원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: 현재 환율: **1 USD = 1,386.92 KRW**  
(기준 시점: 2025년 6월 22일, 실시간 유동성에 따라 소폭 변동 가능)

---

### 🔍 1. **환율 수준 분석**
- **고환율 구간**: 1,380원대는 2022년 10월 이후 **최고 수준**에 근접한 수준입니다.
- **연초 대비**: 2025년 1월 1일(약 1,270원) 대비 **약 9.2% 상승** → 원화 **약 9% 절하**된 상황.
- **심리적 저항선**: 1,400원이 **심리적 마지노선**으로 작용 중. 1,400원 돌파 시 수입 물가 및 물가상승 압력 가중 예상.

---

### 📊 2. **원화 약세의 주요 원인**
| 요인 | 설명 |
|------|------|
| **금리 차이** | 미국 10년물 국채금리 4.2% vs 한국 10년물 3.3% → **차이 0.9%p**로 외국인 자금 이탈 유발 |
| **수출 둔화** | 중국 경기 둔화 + 반도체 가격 하락 → 한국 수출 감소 → 달러 수급 악화 |
| **에너지 가격** | WTI 유가 배럴당 $80+ → 에너지 수입 증가 → 달러 수요 증가 |
| **정치적 불확실성** | 내년 4월 총선 앞두고 정책 불확실성 ↑ → 외국인 자금 회피 심리 |
| **엔캐리지** | 일본 엔화도 약세(USD/JPY 159+) → 아시아 통화 전반 약세 → 원화도 연쇄 하락 |

---

### 📈 3. **향후 전망 (3개월 내)**
| 시나리오 | 환율 범위 | 가능성 | 설명 |
|----------|------------|--------|------|
| **베이스** | 1,370~1,410원 | 50% | 미국 1~2회 추가 인상, 한국은 동결 → 금리차 유지 |
| **상승** | 1,420~1,460원 | 30% | 중국 경기 추가 둔화 + 유가 재상승 → 달러 수요 ↑ |
